# setup Tensorflow

In [19]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.19.0


# Load the dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by 255.0. This also converts the sample data from integers to floating-point numbers:

In [20]:
mnist = tf.keras.datasets.fashion_mnist.load_data()

In [21]:
(x_train, y_train), (x_test, y_test) = mnist
x_train, x_test = x_train / 255.0, x_test / 255.0

In [22]:
x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

# Build a machine learning model

Build a [tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model:

In [23]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10),
    ]
)

[Sequential](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [24]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.65976524,  0.13764226,  0.30031103, -0.00913855, -1.3191959 ,
        -0.7686889 ,  0.6613932 ,  0.3571596 , -0.1203772 ,  0.3786547 ]],
      dtype=float32)

The [tf.nn.softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) function converts these logits to probabilities for each class:

In [25]:
tf.nn.softmax(predictions).numpy()

array([[0.04946942, 0.10981116, 0.12920895, 0.09482009, 0.02558289,
        0.04436412, 0.18539953, 0.1367671 , 0.08483793, 0.13973875]],
      dtype=float32)

**Note:** It is possible to bake the tf.nn.softmax function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using [losses.SparseCategoricalCrossentropy:](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy)

In [26]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to - `tf.math.log(1/10) ~= 2.3`.

In [27]:
loss_fn(y_train[:1], predictions).numpy()

1.9679807

Before you start training, configure and compile the model using Keras [Model.compile](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile). Set the [optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the loss to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [28]:
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])

# Train and evaluate your model

Use the [Model.fit](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) method to adjust your model parameters and minimize the loss

In [29]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


2025-07-09 12:38:19.977898: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7589 - loss: 0.6932
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8495 - loss: 0.4073
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8669 - loss: 0.3655
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8706 - loss: 0.3491
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8780 - loss: 0.3323


The [Model.evaluate](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate) method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [30]:
model.evaluate(x_test, y_test, verbose=2)

2025-07-09 12:39:05.613120: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31360000 exceeds 10% of free system memory.


313/313 - 1s - 2ms/step - accuracy: 0.8761 - loss: 0.3512


[0.35118064284324646, 0.8761000037193298]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [31]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [32]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[8.2997838e-07, 2.2677777e-09, 1.7134681e-07, 1.2674073e-07,
        6.2489036e-08, 1.9643614e-02, 7.6079175e-07, 3.2670215e-02,
        2.4999440e-06, 9.4768178e-01],
       [1.5840502e-05, 1.1939481e-10, 9.9701929e-01, 8.1730107e-08,
        8.7973295e-04, 1.7112896e-10, 2.0850881e-03, 2.5280448e-15,
        1.4160447e-08, 5.4247657e-12],
       [6.1886496e-09, 1.0000000e+00, 4.3953327e-11, 1.3564639e-09,
        1.3079519e-09, 3.1325416e-17, 8.1608338e-11, 1.2508853e-20,
        1.0468450e-10, 9.5611863e-18],
       [2.5754705e-08, 9.9999905e-01, 5.6870153e-10, 1.0111876e-06,
        2.3484704e-08, 1.8309422e-15, 2.2615356e-09, 2.2402877e-17,
        2.0399681e-10, 4.4800610e-15],
       [1.8778515e-01, 2.0462379e-07, 3.4612942e-02, 4.0176590e-03,
        5.3739590e-03, 6.6292415e-07, 7.6288146e-01, 2.7347528e-07,
        5.3244298e-03, 3.2295543e-06]], dtype=float32)>

# Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the guides. If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).